In [1]:
%matplotlib inline

#!unzip './gestos.zip' -d './'
#!pip install tensorboard
#!pip install pytorch-ignite

import numpy as np
import matplotlib.pyplot as plt

import torchvision
import torch
from torchvision import transforms
from torchvision import datasets
import torch.nn as nn
import torch.nn.functional as F

from ignite.engine import Engine, Events
from ignite.metrics import Loss, Accuracy

import time
from torch.utils.tensorboard import SummaryWriter
from ignite.handlers import ModelCheckpoint

import shutil
import os

#os.mkdir('/kaggle/working/models/')

In [ ]:
#Parámetros

bachSize = 32
max_epochs = 200
trainPath = '/kaggle/input/gestos/gestos/train'
validPath = '/kaggle/input/gestos/gestos/valid'

In [ ]:
# Loaders y transformaciones

transform = transforms.Compose(
[
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(0.25),
    transforms.ToTensor()
])

transformVal = transforms.Compose(
[
    transforms.ToTensor()
])

train_dataset = datasets.ImageFolder(trainPath, transform=transform)
valid_dataset = datasets.ImageFolder(validPath, transform=transformVal)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=bachSize, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=256, shuffle=True) 

In [ ]:
# Visualización del dataset de entrenamiento

# Función para mostrar imágenes
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# Obtenemos fotos de 1 batch
dataiter = iter(train_loader)
images, labels = dataiter.next()

# Las mostramos
imshow(torchvision.utils.make_grid(images))

In [ ]:
class HandModel(torch.nn.Module):
    def __init__(self):
        super(type(self), self).__init__()
        self.c1 = nn.Conv2d(3, 32, kernel_size=11, stride=4, padding=2)
        self.c2 = nn.Conv2d(32,96, kernel_size=5, padding=2)
        self.c3 = nn.Conv2d(96,192, kernel_size=3, padding=1)
        self.c4 = nn.Conv2d(192, 128, kernel_size=3, padding=1)
        self.c5 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.ac = nn.ReLU(inplace=True)
        self.mp = nn.MaxPool2d(kernel_size=3, stride=2)
        self.ap = nn.AdaptiveAvgPool2d((3, 3))
        self.f1 = nn.Linear(1152, 512)
        self.f2 = nn.Linear(512, 512)
        self.f3 = nn.Linear(512, 4)
    def forward(self, x):
        x = self.mp(self.ac(self.c1(x)))
        x = self.mp(self.ac(self.c2(x)))
        x = self.ac(self.c3(x))
        x = self.ac(self.c4(x))
        x = self.mp(self.ac(self.c5(x)))
        x = self.ap(x)
        x = torch.flatten(x, 1)
        #display(x.shape)
        #return
        x = self.ac(self.f1(x))
        x = self.ac(self.f2(x))
        x = self.f3(x)
        return x

In [ ]:
# Evaluamos el número de salidas de la última capa
model = HandModel()
display(model)
model.forward(train_dataset[0][0].unsqueeze(0))

In [ ]:
model = HandModel()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss(reduction='sum')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Esto es lo que hace el engine de entrenamiento
def train_one_step(engine, batch):
    optimizer.zero_grad()
    x, y = batch
    x, y = x.to(device), y.to(device)
    yhat = model.forward(x)
    loss = criterion(yhat, y)
    loss.backward()
    optimizer.step()
    return loss.item()

# Esto es lo que hace el engine de evaluación
def evaluate_one_step(engine, batch):
    with torch.no_grad():
        x, y = batch
        x, y = x.to(device), y.to(device)
        yhat = model.forward(x)
        return yhat, y

trainer = Engine(train_one_step)
evaluator = Engine(evaluate_one_step)
metrics = {'Loss': Loss(criterion), 'Acc': Accuracy()}
for name, metric in metrics.items():
    metric.attach(evaluator, name)

In [ ]:
# Envío a tensorboard y checkpoints
with SummaryWriter(log_dir=f'/kaggle/working/') as writer:
    @trainer.on(Events.EPOCH_COMPLETED(every=1)) # Cada 1 epocas
    def log_results(engine):
        evaluator.run(train_loader) 
        writer.add_scalar("train/loss", evaluator.state.metrics['Loss'], engine.state.epoch)
        writer.add_scalar("train/accy", evaluator.state.metrics['Acc'], engine.state.epoch)
        evaluator.run(valid_loader) 
        writer.add_scalar("valid/loss", evaluator.state.metrics['Loss'], engine.state.epoch)
        writer.add_scalar("valid/accy", evaluator.state.metrics['Acc'], engine.state.epoch)
        
        # Guardamos el modelo cada época
        torch.save(model.state_dict(), '/kaggle/working/models/HandModel-'+str(engine.state.epoch)+".pt")

    trainer.run(train_loader, max_epochs=max_epochs)

In [ ]:
# Manejo de archivos

#import shutil
#import os

#os.mkdir('/kaggle/working/models/')
#source = '/kaggle/working/'
#dest1 = '/kaggle/working/models/'
#
#files = os.listdir(source)

#for f in files:
    #shutil.move(source+f, dest1)

In [ ]:
#shutil.make_archive('/kaggle/working/Mod3','zip','/kaggle/working/models')